# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищем месторождения, для каждого определяем значения признаков;
- Строим модель и оцениваем объём запасов;
- Выбираем месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [2]:
#Загрузка необходим библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
#Загрузка датасетов
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

### Подготовка первого датасета

In [4]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
#Проверка на пропуски
data_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [6]:
#Проверка явных дубликатов
data_0.duplicated().sum()

0

In [7]:
#Проверка на корректность типов данных в столбцах
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Удалим столбец, т.к. он несет никакой смысловой нагрузки для обучения

In [8]:
X_0 = data_0.drop(['id', 'product'], axis=1)
y_0 = data_0['product']

In [9]:
X_0.head()

,f0,f1,f2
0,0.705745,-0.497823,1.221170
1,1.334711,-0.340164,4.365080
2,1.022732,0.151990,1.419926
3,-0.032172,0.139033,2.978566
4,1.988431,0.155413,4.751769


In [10]:
y_0.head()

0    105.280062
1     73.037750
2     85.265647
3    168.620776
4    154.036647
Name: product, dtype: float64

In [11]:
X_0_train, X_0_valid, y_0_train, y_0_valid = train_test_split(X_0, y_0, test_size=0.25, random_state=12345)

### Подготовка второго датасета

In [12]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [13]:
#Проверка на пропуски
data_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [14]:
#Проверка явных дубликатов
data_1.duplicated().sum()

0

In [15]:
#Проверка на корректность типов данных в столбцах
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [16]:
X_1 = data_1.drop(['id', 'product'], axis=1)
y_1 = data_1['product']

In [17]:
X_1.head()

,f0,f1,f2
0,-15.001348,-8.276000,-0.005876
1,14.272088,-3.475083,0.999183
2,6.263187,-5.948386,5.001160
3,-13.081196,-11.506057,4.999415
4,12.702195,-8.147433,5.004363


In [18]:
y_1.head()

0      3.179103
1     26.953261
2    134.766305
3    137.945408
4    134.766305
Name: product, dtype: float64

In [19]:
X_1_train, X_1_valid, y_1_train, y_1_valid = train_test_split(X_1, y_1, test_size=0.25, random_state=12345)

### Подготовка третьего датасета

In [20]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [21]:
#Проверка на пропуски
data_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [22]:
#Проверка явных дубликатов
data_1.duplicated().sum()

0

In [23]:
#Проверка на корректность типов данных в столбцах
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [24]:
X_2 = data_2.drop(['id', 'product'], axis=1)
y_2 = data_2['product']

In [25]:
X_2.head()

,f0,f1,f2
0,-1.146987,0.963328,-0.828965
1,0.262778,0.269839,-2.530187
2,0.194587,0.289035,-5.586433
3,2.236060,-0.553760,0.930038
4,-0.515993,1.716266,5.899011


In [26]:
y_2.head()

0     27.758673
1     56.069697
2     62.871910
3    114.572842
4    149.600746
Name: product, dtype: float64

In [27]:
X_2_train, X_2_valid, y_2_train, y_2_valid = train_test_split(X_2, y_2, test_size=0.25, random_state=12345)

Все необходимые данные подготовленны, приступаем к обучению и предсказаниям

## Обучение и проверка модели

In [28]:
#Функция расчета среднего запаса сырья и квадратного корня из средней квадратичной ошибки
def model_lg(X_train, X_valid, y_train, y_valid):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_pred) ** 0.5
    print('Средний запас предсказанного сырья:', pd.Series(y_pred).mean())
    print('RMSE модели:', rmse)
    print('Качество модели (R2):', model.score(X_valid, y_valid))
    return model, pd.Series(y_pred, index=y_valid.index), rmse

### Данные предсказания первого региона

In [29]:
model_0, y_0_pred , rmse_0 =  model_lg(X_0_train, X_0_valid, y_0_train, y_0_valid)

Средний запас предсказанного сырья: 92.59256778438035
RMSE модели: 37.5794217150813
Качество модели (R2): 0.27994321524487786


### Данные предсказания второго региона

In [30]:
model_1, y_1_pred, rmse_1 =  model_lg(X_1_train, X_1_valid, y_1_train, y_1_valid)

Средний запас предсказанного сырья: 68.728546895446
RMSE модели: 0.893099286775617
Качество модели (R2): 0.9996233978805127


### Данные предсказания третьего региона

In [31]:
model_2, y_2_pred, rmse_2 =  model_lg(X_2_train, X_2_valid, y_2_train, y_2_valid)

Средний запас предсказанного сырья: 94.96504596800489
RMSE модели: 40.02970873393434
Качество модели (R2): 0.20524758386040443


Вывод: самые большие предсказанные средние запасы месторождений получились в третьем регионе.

## Подготовка к расчёту прибыли

In [42]:
AMOUNT_OIL_WELLS = 200 #количество скважин в регионе для разработки
DEVELOPMENT_BUDGET = 10_000_000_000 # бюджет на разработку одного региона
PROFIT_PER_UNIT = 450_000 # доход с каждой единицы продукта
print('Достаточный объём сырья для безубыточной разработки новой скважины:',\
      (DEVELOPMENT_BUDGET / AMOUNT_OIL_WELLS / PROFIT_PER_UNIT))

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111


Ни один регион не удовлетворяет минимальному среднему объему запасов сырья на скважину, если взять среднее по всем скважинам, посмотрим что получится, если возьмем 200 лучших.

In [43]:
# Функция расчета прибыли на основе предсказаний по валидационной выборке
def profit(y_true, y_pred, count):
     return round(y_true[y_pred.sort_values(ascending=False).index][:count].sum()\
                  * PROFIT_PER_UNIT - DEVELOPMENT_BUDGET, 0)

In [44]:
profit(y_0_valid, y_0_pred, AMOUNT_OIL_WELLS)

3320826043.0

In [45]:
profit(y_1_valid, y_1_pred, AMOUNT_OIL_WELLS)

2415086697.0

In [46]:
profit(y_2_valid, y_2_pred, AMOUNT_OIL_WELLS)

2710349964.0

Вывод: если исследовать все месторождения в регионах и взять лучших из лучших, то мы легко перекроем затраты и получим неплохую прибыль. Но т.к. у нас бюджет ограничен, посмотрим что будет если взять всего 500 случайных скважин.

## Расчёт прибыли и рисков 

In [65]:
# Bootstrap
def bootstrap(y_valid, y_pred):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        subsample = y_valid.sample(n=500, replace=True, random_state=state)
        probs_subsample = y_pred[subsample.index]
        values.append(profit(subsample, probs_subsample, AMOUNT_OIL_WELLS))

    values=pd.Series(values)
    print('Среднее значение прибыли:', round(values.mean(),0))
    print('Границы 95% интервала', round(values.quantile(.025),0), round(values.quantile(.975),0))
    print('Вероятность убытка:', values[values < 0].count() / len(values))

In [66]:
values_0 = bootstrap(y_0_valid, y_0_pred)

Среднее значение прибыли: 425938527.0
Границы 95% интервала -102090095.0 947976354.0
Вероятность убытка: 0.06


In [67]:
bootstrap(y_1_valid, y_1_pred)

Среднее значение прибыли: 515222773.0
Границы 95% интервала 68873225.0 931547591.0
Вероятность убытка: 0.01


In [68]:
bootstrap(y_2_valid, y_2_pred)

Среднее значение прибыли: 435008363.0
Границы 95% интервала -128880547.0 969706954.0
Вероятность убытка: 0.064


**Общий вывод:** на данных по трем регионам были обучены модели линейной регрессии и получены предсказания на валидационной выборке. По результатам предсказаний получили, что при исследовании всех скважин все три региона выходят на прибыль. После проведения анализа методом Bootstrap, выбирая 500 случайных скважин в регионе и из них 200 лучших, мы видем, что в регионах номер 1 и 3 вероятность убытка более 6% и 6,4%, что не удовлетворяет критериям выбора. Регион номер 2 отвечает критериям неубыточности, объем прибыли, который мы можем получить с данных месторождений от 71 до 893 млн рублей.